## Basic Neural Network Manual Implementation

This is more for illustration that the given features are not sufficient for classification. Notice that this is even without even splitting the data (i.e. using all of the data to both train and test). Also of note is that it ignores class imbalance during training.

In [1]:
#Load preprocessed data
import numpy as np
dataa = np.load("Features.npy")
truth = np.load("true_labels.npy")

In [2]:
#Remove values with nan values
features = dataa[:,~np.isnan(dataa).any(axis=0)]
#4 features had issues calculating with feets

In [3]:
#One-hot encoding of labels
labels = np.zeros((truth.size, truth.max()))
labels[np.arange(truth.size),truth-1] = 1

In [4]:
#Import TF
import tensorflow as tf

In [5]:
#Put labels/features in float32 format, which works better with TF than float64
labels = np.float32(labels)
features = np.float32(features)
#Create placeholders for an unknown number of stars
features_placeholder = tf.placeholder(features.dtype, [None,features.shape[1]])
labels_placeholder = tf.placeholder(labels.dtype, [None, labels.shape[1]])

In [21]:
#Parameters
learning_rate = 0.0005
epoch = 3500
batch_size = 23400
display_step = 100
n_hidden_1 = 118
n_hidden_2 = 118
n_hidden_3 = 59
num_input = 59
num_classes = 17
n_stars = len(truth)

In [15]:
#Create an iterator to use data in TF
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=100)
iterator = dataset.make_initializable_iterator()

In [16]:
#Weights and biases for the TF model (3 hidden layers)
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [17]:
def neural_net(x):
    #Hidden fully connected layers
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))
    #Output fully connected layer with a neuron for each class
    #Do not apply softmax, the loss function directly does this
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

In [18]:
#Get features and feed through the neural net
feats, labs = iterator.get_next()
logits = neural_net(feats)

In [19]:
#Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=labs))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#Evaluate model accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labs, 1))
#Get number predicted correctly and prediction average
accuracy = tf.reduce_sum(tf.cast(correct_pred, tf.float32))
accuracy2 = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Variable initializer
init = tf.global_variables_initializer()

In [22]:
# Start training
with tf.Session() as sess:
    #Epoch
    ep = 0
    #Initialize variables (weights/biases)
    sess.run(init)
    
    #Run training
    for i in range(epoch):
        #Initialize the iterator to run through the data
        sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                                  labels_placeholder: labels})
        #accuracy placeholder
        acc = 0
        #This loop runs on training data, breaking when it runs out of batches to feed in (finishing the epoch)
        while True:
            try:
                _,tempp = sess.run([train_op,accuracy])
                acc += tempp
            except tf.errors.OutOfRangeError:
                break
        ep += 1
        if ep % display_step == 0 or ep == 1:
            print("Epoch",ep, "Prediction Accuracy: ", acc/(n_stars - n_stars % batch_size))
    #print("Testing Accuracy:", \
    #sess.run(accuracy, feed_dict={features_placeholder: features,
    #                              labels_placeholder: labels}))

Epoch 1 Prediction Accuracy:  0.010807116464834156
Epoch 100 Prediction Accuracy:  0.6528694389269772
Epoch 200 Prediction Accuracy:  0.6535315349949808
Epoch 300 Prediction Accuracy:  0.653616966745691
Epoch 400 Prediction Accuracy:  0.6536383246833686
Epoch 500 Prediction Accuracy:  0.6537451143717563
Epoch 600 Prediction Accuracy:  0.6537451143717563
Epoch 700 Prediction Accuracy:  0.6536596826210461
Epoch 800 Prediction Accuracy:  0.6537451143717563
Epoch 900 Prediction Accuracy:  0.6536383246833686
Epoch 1000 Prediction Accuracy:  0.6537451143717563


KeyboardInterrupt: 